This notebook is dedicated to clean the data and export the clean version of each dataset in a separate csv file.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Police violence

In [2]:
PV_data = pd.read_excel('./data/MPVDatasetDownload.xlsx')
PV_data.head()

,Victim's name,Victim's age,Victim's gender,Victim's race,URL of image of victim,Date of Incident (month/day/year),Street Address of Incident,City,State,Zipcode,...,Unarmed/Did Not Have an Actual Weapon,Alleged Weapon (Source: WaPo and Review of Cases Not Included in WaPo Database),Alleged Threat Level (Source: WaPo),Fleeing (Source: WaPo),Body Camera (Source: WaPo),WaPo ID (If included in WaPo database),Off-Duty Killing?,Geography (via Trulia methodology based on zipcode population density: http://jedkolko.com/wp-content/uploads/2015/05/full-ZCTA-urban-suburban-rural-classification.xlsx ),MPV ID,Fatal Encounters ID
0,Pedro Martinez,66,Male,Hispanic,NaN,2020-11-18,NaN,Tyler,TX,NaN,...,Allegedly Armed,unknown weapon,other,not fleeing,no,6321.0,NaN,NaN,NaN,NaN
1,James Horton,42,Male,NaN,NaN,2020-11-17,NaN,Hart County,KY,NaN,...,Allegedly Armed,gun,attack,other,no,6322.0,NaN,NaN,NaN,NaN
2,Dustin James Acosta,28,Male,White,NaN,2020-11-17,NaN,St. Augustine,FL,NaN,...,Allegedly Armed,gun,attack,NaN,No,6320.0,NaN,NaN,NaN,NaN
3,Javon Brice,39,Male,NaN,NaN,2020-11-17,NaN,La Vergne,TN,NaN,...,Allegedly Armed,gun,attack,car,no,6319.0,NaN,NaN,NaN,NaN
4,Joshua D. Evans,32,Male,NaN,NaN,2020-11-16,NaN,Batesville,IN,NaN,...,Allegedly Armed,gun,attack,not fleeing,no,6314.0,NaN,NaN,NaN,NaN


In [3]:
PV_data.columns

Index(['Victim's name', 'Victim's age', 'Victim's gender', 'Victim's race',
       'URL of image of victim', 'Date of Incident (month/day/year)',
       'Street Address of Incident', 'City', 'State', 'Zipcode', 'County',
       'Agency responsible for death', 'ORI Agency Identifier (if available)',
       'Cause of death',
       'A brief description of the circumstances surrounding the death',
       'Official disposition of death (justified or other)',
       'Criminal Charges?',
       'Link to news article or photo of official document',
       'Symptoms of mental illness?', 'Unarmed/Did Not Have an Actual Weapon',
       'Alleged Weapon (Source: WaPo and Review of Cases Not Included in WaPo Database)',
       'Alleged Threat Level (Source: WaPo)', 'Fleeing (Source: WaPo)',
       'Body Camera (Source: WaPo)', 'WaPo ID (If included in WaPo database)',
       'Off-Duty Killing?',
       'Geography (via Trulia methodology based on zipcode population density: http://jedkolko.com/wp-co

We drop columns that won't be used, or that don't contain enough data. Then we rename them to be more readable and easier to handle.

In [4]:
PV_data.drop(["Victim's name",
           "URL of image of victim", "Street Address of Incident", "Zipcode", "County", "ORI Agency Identifier (if available)",
           "Agency responsible for death", "Cause of death",  
           "A brief description of the circumstances surrounding the death",
           "Official disposition of death (justified or other)",
           "Link to news article or photo of official document", "Symptoms of mental illness?",
           "Alleged Threat Level (Source: WaPo)", #3968 are attack, the rest are undetermined/other
           "Body Camera (Source: WaPo)", "WaPo ID (If included in WaPo database)",
           "Off-Duty Killing?", #239/8629 are not NaN
           "Geography (via Trulia methodology based on zipcode population density: http://jedkolko.com/wp-content/uploads/2015/05/full-ZCTA-urban-suburban-rural-classification.xlsx )",
           "MPV ID", "Fatal Encounters ID"], 
          inplace=True, axis=1)

In [5]:
PV_data = PV_data.rename(
    columns={
        "Victim's age": 'victim_age',
        "Victim's gender": 'victim_gender',
        "Victim's race": 'victim_race',
        "Date of Incident (month/day/year)": 'date',                              
        "Criminal Charges?": 'criminal_charges',
        "Unarmed/Did Not Have an Actual Weapon": 'victim_armed',
        "Alleged Weapon (Source: WaPo and Review of Cases Not Included in WaPo Database)": 'alleged_weapon',
        "Alleged Threat Level (Source: WaPo)": 'alleged_threat_level',
        "Fleeing (Source: WaPo)": 'victim_fleeing'
    })

We explore the data inside a number of features. We make some changes to handle missing data, unwanted types, repetitive data, etc ...

In [6]:
PV_data['victim_race'].unique()

array(['Hispanic', nan, 'White', 'Unknown race', 'Black', 'Unknown Race',
       'Native American', 'unknown race', 'Asian', 'Pacific Islander'],
      dtype=object)

In [7]:
#we are only interested in "white", "black" and "hispanic" people
PV_data['victim_race'] = PV_data['victim_race'].str.lower()
races = ["white", "black", "hispanic"]
PV_data = PV_data.query('victim_race in @races')
PV_data['victim_race'].unique()

array(['hispanic', 'white', 'black'], dtype=object)

In [8]:
PV_data['criminal_charges'].unique()

array(['No known charges', 'Charged with a crime',
       'Charged, Convicted, Sentenced to 7 years in prison',
       'Charged, Acquitted',
       'Charged, Convicted, Sentenced to 1 year in prison and 4 years probation',
       'Charged, Charges Dropped', 'Charged, Convicted',
       'Charged, Plead No Contest, Sentenced to 10 years probation',
       'Charged, Convicted, Sentenced to 6 years in prison',
       'Charged, Convicted of reckless homicide, sentenced to 1 year in prison',
       'Charged, Convicted, Sentenced to 14 years in prison',
       'Charged, Convicted, Sentenced to 5 years in prison',
       'Charged, Convicted, Sentenced to 40 years in prison',
       'Charged, Convicted, Sentenced to 25 years in prison',
       'Charged, Convicted, Sentenced to 3 months in jail',
       'Charged, Mistrial',
       'Charged, Convicted, Sentenced to 30 years in prison',
       'Charged, Convicted, Sentenced to 2.5 years in prison',
       'Charged, Mistrial, Plead Guilty to Civil 

In [9]:
PV_data.loc[PV_data['criminal_charges'] != "No known charges", 'criminal_charges'] = "Charged"
PV_data['criminal_charges'].unique()

array(['No known charges', 'Charged'], dtype=object)

In [10]:
PV_data['victim_armed'].unique()

array(['Allegedly Armed', 'Unclear', 'Vehicle',
       'Unarmed/Did Not Have an Actual Weapon'], dtype=object)

In [11]:
PV_data.loc[PV_data['victim_armed'] == "Unarmed/Did Not Have an Actual Weapon", 'victim_armed'] = "Unarmed"
PV_data['victim_armed'].unique()

array(['Allegedly Armed', 'Unclear', 'Vehicle', 'Unarmed'], dtype=object)

In [12]:
PV_data['victim_fleeing'].unique()

array(['not fleeing', nan, 'car', 'foot', 'Not fleeing', 'other', 'Foot',
       'Car', 'Not Fleeing', 'Other'], dtype=object)

In [13]:
PV_data['victim_fleeing']=PV_data['victim_fleeing'].str.lower()
PV_data['victim_fleeing'].unique()

array(['not fleeing', nan, 'car', 'foot', 'other'], dtype=object)

In [14]:
PV_data.query('victim_age == "Unknown"')
#266 victims are of "unknown" age

,victim_age,victim_gender,victim_race,date,City,State,criminal_charges,victim_armed,alleged_weapon,victim_fleeing
252,Unknown,Male,white,2020-08-18,Bay City,TX,No known charges,Allegedly Armed,gun,foot
253,Unknown,Male,hispanic,2020-08-17,Bakersfield,CA,No known charges,Allegedly Armed,knife,car
258,Unknown,Male,white,2020-08-16,Las Vegas,NV,No known charges,Allegedly Armed,gun,not fleeing
286,Unknown,Male,black,2020-08-08,Bossier City,LA,No known charges,Allegedly Armed,knife,not fleeing
287,Unknown,Male,hispanic,2020-08-07,Rancho Mirage,CA,No known charges,Allegedly Armed,knife,not fleeing
...,...,...,...,...,...,...,...,...,...,...
7517,Unknown,Male,white,2014-01-08,New Orleans,LA,No known charges,Allegedly Armed,gun,NaN
7818,Unknown,Male,hispanic,2013-09-18,Los Angeles,CA,No known charges,Allegedly Armed,gun,NaN
7836,Unknown,Male,white,2013-09-09,Houston,TX,No known charges,Allegedly Armed,gun,NaN
7951,Unknown,Male,hispanic,2013-08-05,Houston,TX,No known charges,Allegedly Armed,gun,NaN


In [15]:
PV_data.loc[PV_data['victim_age'] == "Unknown",  'victim_age'] = np.NaN

In [16]:
PV_data['victim_age'].unique()

array([66, 28, 35, 38, 40, 18, 16, 37, 25, 56, 34, 36, 30, 26, 39, 43, 21,
       44, 20, 27, 53, 19, 33, nan, 45, 61, 48, 32, 57, 46, 47, 23, 29,
       52, 51, 24, 41, 62, 31, 42, 22, 55, 88, 50, 17, 60, 63, 73, 49, 68,
       58, 65, 69, 64, 54, 59, 67, 82, 75, 15, 70, 74, 77, 76, 80, 14, 71,
       6, 72, 84, 1, 81, 78, '40s', 79, 91, 83, 13, 10, 12, 86, 95, 5, 93,
       7, 107, 89], dtype=object)

In [17]:
PV_data.query('victim_age == "40s"')

,victim_age,victim_gender,victim_race,date,City,State,criminal_charges,victim_armed,alleged_weapon,victim_fleeing
3502,40s,Male,black,2017-09-28,Oakland,CA,No known charges,Unarmed,no object,foot


In [18]:
PV_data.loc[PV_data['victim_age'] == "40s", 'victim_age'] = 40 #Only one record is of age '40s', so we allow ourselves to change that into 40

In [19]:
PV_data.alleged_weapon.unique()

array(['unknown weapon', 'gun', 'undetermined', 'vehicle', 'knife',
       'curtain rod', 'machete', 'no object', 'shovel', 'toy weapon',
       'pepper spray', 'bottle', 'sword', 'baseball bat', 'Taser',
       'hammer', 'vehicle and gun', 'rock', 'samurai sword',
       'sharp object', 'gun and car', 'windshield wiper', 'metal pipe',
       'gun and vehicle', 'pick-axe', 'tire iron', 'screwdriver',
       'BB gun', 'meat cleaver', 'chair', 'chain', 'hatchet',
       'car, knife and mace', 'vehicle and machete', 'box cutter', 'ax',
       'crowbar', 'scissors', 'baseball bat and knife',
       'straight edge razor', 'baton', 'Airsoft pistol', 'air pistol',
       'wasp spray', 'BB gun and vehicle', 'piece of wood',
       'gun and knife', 'garden tool', 'barstool', 'wrench', 'flag pole',
       'lawn mower blade', 'crossbow', 'lamp', 'incendiary device',
       'pipe', 'pole and knife', 'pitchfork', 'gun and sword', 'taser',
       'blunt object', 'chainsaw', 'fireworks', 'baseball ba

In [20]:
import re #RegEx

#We are interested in guns
PV_data['alleged_weapon'] = PV_data['alleged_weapon'].replace(to_replace ='.*gun.*', value = 'gun', regex = True)

Result of the cleaning

In [21]:
PV_data.head()

,victim_age,victim_gender,victim_race,date,City,State,criminal_charges,victim_armed,alleged_weapon,victim_fleeing
0,66,Male,hispanic,2020-11-18,Tyler,TX,No known charges,Allegedly Armed,unknown weapon,not fleeing
2,28,Male,white,2020-11-17,St. Augustine,FL,No known charges,Allegedly Armed,gun,NaN
7,35,Male,white,2020-11-14,Meredith,NH,No known charges,Unclear,undetermined,NaN
8,38,Male,hispanic,2020-11-14,Sacramento,CA,No known charges,Allegedly Armed,gun,NaN
10,40,Male,white,2020-11-14,Cantonment,FL,No known charges,Vehicle,vehicle,car


Finally, we want to keep our data sorted by race

In [22]:
PV_data = PV_data.sort_values(by='victim_race')

Exporting the daraframe to csv to be used in other notebooks

In [23]:
PV_data.to_csv('./data/police_violence_full_date.csv')

Most of the plots will be using years only (instead of full date), so we decide to change that here and export the dataset with years only in a new csv file to avoid cleaning that in the processing steps.

In [24]:
PV_data['year'] = pd.DatetimeIndex(PV_data['date']).year #changes dates to years
PV_data.drop(['date'], inplace=True, axis=1)

In [25]:
PV_data.to_csv('./data/police_violence.csv')

***

# Ranking of states by gun laws strength:

In [26]:
url = r'https://giffords.org/lawcenter/resources/scorecard/#rankings'
tables = pd.read_html(url) # Returns list of all tables on page
gunlaw_ranking = tables[0] # Select table of interest

gunlaw_ranking.head()

,Gun LawStrength(Ranked),State,2019Grade,Gun DeathRate(Ranked),Gun DeathRate(Per 100K)
0,38,Alabama,F,2,21.70
1,42,Alaska,F,7,20.74
2,45,Arizona,F,18,15.29
3,40,Arkansas,F,8,18.96
4,1,California,A,44,7.45


We rename the columns.

In [27]:
gunlaw_ranking = gunlaw_ranking.rename(
    columns={
        "Gun LawStrength(Ranked)": 'state_gunlaw_strength',
        "2019Grade": 'state_gunlaw_grade',
        "Gun DeathRate(Ranked)": 'state_gun_death_rate',
        "Gun DeathRate(Per 100K)": 'state_gun_death_rate_per_100K'
    })

We use a dictionary containing the full names of states as keys, and their abbreviations as values; to change the names into abbreviations in this data to make it compatible with the other datasets.

In [28]:
us_state_abbrev = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'American Samoa': 'AS',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'District of Columbia': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Guam': 'GU',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Northern Mariana Islands':'MP',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Puerto Rico': 'PR',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virgin Islands': 'VI',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY'
}

In [29]:
gunlaw_ranking = gunlaw_ranking.replace({"State": us_state_abbrev})

Since the dataset represents ranking of the states, we are interested in keeping it sorted by state's gun law strength (i.e. grade).

In [30]:
gunlaw_ranking.sort_values(by=['state_gunlaw_strength'])

,state_gunlaw_strength,State,state_gunlaw_grade,state_gun_death_rate,state_gun_death_rate_per_100K
4,1,CA,A,44,7.45
29,2,NJ,A,46,4.75
6,3,CT,A-,45,4.91
31,4,NY,A-,47,4.03
10,5,HI,A-,48,4.03
19,6,MD,A-,33,11.61
20,7,MA,A-,49,3.46
12,8,IL,A-,36,10.78
38,9,RI,B+,50,3.28
46,10,WA,B+,38,10.32


Exporting the dataframe to csv

In [31]:
gunlaw_ranking.to_csv('./data/gunlaw_ranking.csv')

***